In [44]:
from src.datasets.ukbiobank_dataset import UkBiobankDatasetArgs
from src.experiments.multi_ds_vessel_experiment import MultiDSVesselExperimentArgs
from src.args.yaml_config import YamlConfig
from src.models.auto_sam_model import AutoSamModel
from torch.utils.data import DataLoader
from src.datasets.ukbiobank_dataset import UkBiobankDataset
from typing import cast
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np


from src.datasets.base_dataset import Batch

import torch

# Mask output path
iteration = 0
output_path = f"/dhc/dsets/retina_masks/v{iteration}"

# Setup configs
yaml_config = YamlConfig().config
experiment_config = MultiDSVesselExperimentArgs(
    sam_model='vit_b', 
    experiment_id='label_generation', 
    prompt_encoder_checkpoint='/dhc/groups/mp2024cl2/results/drive_experiment/2024-11-15_11#24#47/prompt_encoder.pt',
    sam_checkpoint='/dhc/groups/mp2024cl2/sam_vit_b.pth',
    hard_net_cp='/dhc/groups/mp2024cl2/hardnet68.pth',
    batch_size=1,
)
ds_config = UkBiobankDatasetArgs()

In [45]:
# Load pretrained model
model = AutoSamModel(experiment_config)
if experiment_config.prompt_encoder_checkpoint is not None:
    print(
        f"loading prompt-encoder model from checkpoint {experiment_config.prompt_encoder_checkpoint}"
    )
    model.prompt_encoder.load_state_dict(
        torch.load(experiment_config.prompt_encoder_checkpoint, map_location="cuda"),
        strict=True,
    )
model.to("cuda")

/dhc/home/leon.hermann/MP/mp-lippert-experiments/src/models/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)
/dhc/

ImageNet pretrained weights for HarDNet68 is loaded
loading prompt-encoder model from checkpoint /dhc/groups/mp2024cl2/results/drive_experiment/2024-11-15_11#24#47/prompt_encoder.pt


AutoSamModel(
  (sam): SamBatched(
    (image_encoder): ImageEncoderViT(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (blocks): ModuleList(
        (0-11): 12 x Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (lin1): Linear(in_features=768, out_features=3072, bias=True)
            (lin2): Linear(in_features=3072, out_features=768, bias=True)
            (act): GELU(approximate='none')
          )
        )
      )
      (neck): Sequential(
        (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): LayerNorm2d()
        (2): Conv2d(256, 256, kernel_size=(3, 3)

In [50]:
import cv2
from pathlib import Path

# Create output folder if it does not exist
Path(output_path).mkdir(parents=True, exist_ok=True)

filtered_samples_filepath = ds_config.filter_scores_filepath

with open(filtered_samples_filepath, 'r') as f:
    for sample in tqdm(f.readlines()[1:]):
        file_p, neg_prob, pos_prob, label = sample.strip().split(',')
        
        if label == '1':
            mask_out_path = f"{output_path}/{file_p.split('/')[-1]}"
            image, mask  = model.segment_image_from_file(file_p)
            mask[mask > 255 / 2] = 255
            mask[mask <= 255 / 2] = 0
            mask_img = mask * np.array([1, 1, 1])
            cv2.imwrite(mask_out_path, mask_img)
        

  0%|          | 0/168247 [00:00<?, ?it/s]

  0%|          | 372/168247 [00:37<4:39:32, 10.01it/s]


KeyboardInterrupt: 